In [1]:
import pandas as pd

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [5]:
PMB_path='/content/PMBookextraction.csv'
PMI_path='/content/extration_result2.csv'

In [6]:
df1=pd.read_csv(PMB_path)
df2=pd.read_csv(PMI_path)

In [7]:
df1['content'][70]

'11.7.1.1 PROJECT MANAGEM ENT PLAN\nDescribed in Section 4.2.3.1. Project management plan components include but are not limited to the risk \nmanagement plan. Described in Section 11.3.1.1. The risk management plan provides guidance on how and when \nrisks should be reviewed, which policies and procedures should be followed, the roles and responsibilities in the monitoring process, and reporting formats.\n11.7.1.2'

In [8]:
df2['content'][70]

"['7.4 Documenting the Results of the Perform Quantitative Risk Analysis Process The contingency reserves calculated in quantitative project cost and schedule risk analysis are incorporated, respectively, into the cost estimate and the schedule to establish a prudent target and a realistic expectation for the project. Contingency reserves may also be established to provide for the capture of opportunities that are judged to be priorities for the project. If the contingency reserve required exceeds the time or resources available, changes in the project scope and plan may result. Also, the results of the analysis may provide more or less urgency to risk response (see Plan Risk Responses, Chapter 8) depending on the probability of achieving the plan’s objectives or the amount of contingency reserve required to provide the necessary level of conﬁ dence. The results of a quantitative risk analysis are recorded and passed on to the person and/or group responsible for project management with

In [9]:
import re
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')

# Define a function to clean and preprocess text
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    text = re.sub(r'\d', '', text)  # Remove numbers
    text = ' '.join([word for word in text.split() if word not in stopwords.words('english')])  # Remove stopwords
    return text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [10]:
# Apply the preprocessing function to both dataframes
df1['content'] = df1['content'].apply(preprocess_text)
df2['content'] = df2['content'].apply(preprocess_text)

In [11]:
df2['content'][70]

'documenting results perform quantitative risk analysis process contingency reserves calculated quantitative project cost schedule risk analysis incorporated respectively cost estimate schedule establish prudent target realistic expectation project contingency reserves may also established provide capture opportunities judged priorities project contingency reserve required exceeds time resources available changes project scope plan may result also results analysis may provide less urgency risk response see plan risk responses chapter depending probability achieving plans objectives amount contingency reserve required provide necessary level conﬁ dence results quantitative risk analysis recorded passed person andor group responsible project management within organization actions required make full use results chapter'

In [12]:
pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 56.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 39.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 106.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 79.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 37.9 MB/s eta 0:00:00


In [13]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from transformers import AutoTokenizer, AutoModel



In [14]:
#tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
#df1_tokens = df1["content"].apply(lambda x: tokenizer.encode(x, add_special_tokens=True))
#df2_tokens = df2["content"].apply(lambda x: tokenizer.encode(x, add_special_tokens=True))


In [15]:
from transformers import AutoTokenizer, AutoModel
import numpy as np

# Define a function to encode and concatenate text segments
def encode_and_concatenate(text, max_sequence_length):
    tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
    tokens = tokenizer.encode(text, add_special_tokens=True, return_tensors='pt', max_length=max_sequence_length, truncation=True)
    return tokens

# Set the maximum sequence length
max_sequence_length = 512

# Encode and concatenate text segments for df1
df1_tokens = df1["content"].apply(lambda x: encode_and_concatenate(x, max_sequence_length))

# Encode and concatenate text segments for df2
df2_tokens = df2["content"].apply(lambda x: encode_and_concatenate(x, max_sequence_length))

# Now, you have tokenized and encoded segments for both dataframes (df1_encoded_tokens and df2_encoded_tokens)


In [16]:
file1_text = [' '.join(map(str, tokenized)) for tokenized in df1_tokens]
file2_text = [' '.join(map(str, tokenized)) for tokenized in df2_tokens]

tfidf_vectorizer = TfidfVectorizer()


df1_tfidf_matrix = tfidf_vectorizer.fit_transform(file1_text)
df2_tfidf_matrix = tfidf_vectorizer.transform(file2_text)



In [17]:
cosine_similarities = linear_kernel(df1_tfidf_matrix, df2_tfidf_matrix)


In [18]:
for i in range(len(df1)):
    for j in range(len(df2)):
        similarity_score = cosine_similarities[i][j]
        if similarity_score > 0.5:
            print(f"Content {i+1} in PMBOOK is similar to Content {j+1} in PMI with a similarity score of {similarity_score:.2f}")



Content 9 in PMBOOK is similar to Content 16 in PMI with a similarity score of 0.53
Content 9 in PMBOOK is similar to Content 17 in PMI with a similarity score of 0.58
Content 24 in PMBOOK is similar to Content 47 in PMI with a similarity score of 0.51
Content 25 in PMBOOK is similar to Content 17 in PMI with a similarity score of 0.53
Content 63 in PMBOOK is similar to Content 17 in PMI with a similarity score of 0.57


In [19]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_similarities = cosine_similarity(df1_tfidf_matrix, df2_tfidf_matrix)

for i in range(len(df1)):
    for j in range(len(df2)):
        similarity_score = cosine_similarities[i][j]
        print(f"Content {i+1} in File 1 is similar to Content {j+1} in File 2 with a cosine similarity score of {similarity_score:.2f}")


Streaming output truncated to the last 5000 lines.
Content 36 in File 1 is similar to Content 30 in File 2 with a cosine similarity score of 0.21
Content 36 in File 1 is similar to Content 31 in File 2 with a cosine similarity score of 0.15
Content 36 in File 1 is similar to Content 32 in File 2 with a cosine similarity score of 0.12
Content 36 in File 1 is similar to Content 33 in File 2 with a cosine similarity score of 0.13
Content 36 in File 1 is similar to Content 34 in File 2 with a cosine similarity score of 0.13
Content 36 in File 1 is similar to Content 35 in File 2 with a cosine similarity score of 0.13
Content 36 in File 1 is similar to Content 36 in File 2 with a cosine similarity score of 0.14
Content 36 in File 1 is similar to Content 37 in File 2 with a cosine similarity score of 0.10
Content 36 in File 1 is similar to Content 38 in File 2 with a cosine similarity score of 0.06
Content 36 in File 1 is similar to Content 39 in File 2 with a cosine similarity score of 0.04

In [20]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_similarities = cosine_similarity(df1_tfidf_matrix, df2_tfidf_matrix)

for i in range(len(df1)):
    max_similarity_score = max(cosine_similarities[i])
    max_similar_content_index = list(cosine_similarities[i]).index(max_similarity_score)
    print(f"Content {i+1} in File 1 is most similar to Content {max_similar_content_index+1} in File 2 with a cosine similarity score of {max_similarity_score:.2f}")


Content 1 in File 1 is most similar to Content 101 in File 2 with a cosine similarity score of 0.20
Content 2 in File 1 is most similar to Content 17 in File 2 with a cosine similarity score of 0.49
Content 3 in File 1 is most similar to Content 10 in File 2 with a cosine similarity score of 0.38
Content 4 in File 1 is most similar to Content 21 in File 2 with a cosine similarity score of 0.27
Content 5 in File 1 is most similar to Content 19 in File 2 with a cosine similarity score of 0.24
Content 6 in File 1 is most similar to Content 20 in File 2 with a cosine similarity score of 0.23
Content 7 in File 1 is most similar to Content 10 in File 2 with a cosine similarity score of 0.31
Content 8 in File 1 is most similar to Content 23 in File 2 with a cosine similarity score of 0.40
Content 9 in File 1 is most similar to Content 17 in File 2 with a cosine similarity score of 0.58
Content 10 in File 1 is most similar to Content 17 in File 2 with a cosine similarity score of 0.37
Content 

In [21]:
import torch

In [22]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModel.from_pretrained("bert-base-uncased")

file1_tokens = df1["content"].apply(lambda x: tokenizer.encode(x, add_special_tokens=True))
file2_tokens = df2["content"].apply(lambda x: tokenizer.encode(x, add_special_tokens=True))


Token indices sequence length is longer than the specified maximum sequence length for this model (657 > 512). Running this sequence through the model will result in indexing errors


In [23]:
file1_text = df1["content"].tolist()
file2_text = df2["content"].tolist()

In [24]:
def get_bert_embeddings(texts):
    inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt", max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    embeddings = torch.mean(outputs.last_hidden_state, dim=1)
    return embeddings

file1_embeddings = torch.cat([get_bert_embeddings(text) for text in df1["content"]])
file2_embeddings = torch.cat([get_bert_embeddings(text) for text in df2["content"]])


In [25]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_similarities = cosine_similarity(file1_embeddings, file2_embeddings)


In [26]:

df1['PMI_content']=''
df1['PMI_chapter']=''
df1['PMI_subtitle']=''
for i in range(len(df1)):
    max_similarity_score = max(cosine_similarities[i])
    max_similar_content_index = list(cosine_similarities[i]).index(max_similarity_score)
    print(f"Content {i+1} in PMBOK  is most similar to Content {max_similar_content_index+1} in PMI with a cosine similarity score of {max_similarity_score:.2f}")
    df1['PMI_content'][i]=df2['content'][max_similar_content_index]
    df1['PMI_chapter'][i]=df2['Chapter'][max_similar_content_index]
    df1['PMI_subtitle'][i]=df2['subTitle'][max_similar_content_index]

Content 1 in PMBOK  is most similar to Content 95 in PMI with a cosine similarity score of 0.85
Content 2 in PMBOK  is most similar to Content 24 in PMI with a cosine similarity score of 0.92
Content 3 in PMBOK  is most similar to Content 13 in PMI with a cosine similarity score of 0.93
Content 4 in PMBOK  is most similar to Content 104 in PMI with a cosine similarity score of 0.89
Content 5 in PMBOK  is most similar to Content 35 in PMI with a cosine similarity score of 0.89
Content 6 in PMBOK  is most similar to Content 35 in PMI with a cosine similarity score of 0.91
Content 7 in PMBOK  is most similar to Content 67 in PMI with a cosine similarity score of 0.89
Content 8 in PMBOK  is most similar to Content 23 in PMI with a cosine similarity score of 0.95
Content 9 in PMBOK  is most similar to Content 17 in PMI with a cosine similarity score of 0.97
Content 10 in PMBOK  is most similar to Content 8 in PMI with a cosine similarity score of 0.91
Content 11 in PMBOK  is most similar to

<ipython-input-26-2fd17632fe17>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['PMI_content'][i]=df2['content'][max_similar_content_index]
<ipython-input-26-2fd17632fe17>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['PMI_chapter'][i]=df2['Chapter'][max_similar_content_index]
<ipython-input-26-2fd17632fe17>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['PMI_subtitle'][i]=df2['subTitle'][max_similar_content_index]


In [27]:
print(cosine_similarities)

[[0.6693837  0.7628286  0.679475   ... 0.8467394  0.78012884 0.7269829 ]
 [0.79067737 0.85907847 0.78723234 ... 0.8368893  0.8638886  0.7011629 ]
 [0.78419566 0.88101023 0.8069027  ... 0.87674487 0.8612752  0.7342092 ]
 ...
 [0.7142458  0.7986531  0.72034687 ... 0.8041381  0.81025314 0.70594335]
 [0.87203217 0.91516745 0.89198315 ... 0.7983364  0.85734737 0.70433855]
 [0.72650564 0.8346542  0.73080933 ... 0.8385756  0.8806458  0.72304034]]


In [28]:
df1.to_csv('/content/Enriched.csv')